In [3]:
!pip install ridgeplot

  Obtaining dependency information for ridgeplot from https://files.pythonhosted.org/packages/c4/83/3ce3c7e02351506abb34fae3541cca9a8e2b1a8e1ace6447deb492d6953e/ridgeplot-0.1.23-py3-none-any.whl.metadata
  Obtaining dependency information for plotly>=4 from https://files.pythonhosted.org/packages/df/79/c80174d711ee26ee5da55a9cc3e248f1ec7a0188b5e4d6bbbbcd09b974b0/plotly-5.17.0-py2.py3-none-any.whl.metadata
     ---------------------------------------- 9.2/9.2 MB 15.1 MB/s eta 0:00:00
  Obtaining dependency information for tenacity>=6.2.0 from https://files.pythonhosted.org/packages/f4/f1/990741d5bb2487d529d20a433210ffa136a367751e454214013b441c4575/tenacity-8.2.3-py3-none-any.whl.metadata
     ------------------------------------- 233.8/233.8 kB 14.9 MB/s eta 0:00:00
   ---------------------------------------- 47.4/47.4 kB ? eta 0:00:00
   ---------------------------------------- 15.6/15.6 MB 36.4 MB/s eta 0:00:00


In [15]:
import ridgeplot as rp
import plotly
import numpy as np
import pandas as pd
import requests
from dataclasses import dataclass
from enum import Enum
import time



In [25]:
# each function is a conversion from the unit to DALY/dollar.
class Units(Enum):
    daly_over_dollar = lambda x: x
    dollar_over_daly = lambda x: 1/x
    in_gd_daly_over_dollar = lambda x: x/(0.00335*2.5) # GW's estimate of GiveDirectly's impact is 0.00335 doubling of consumption per dollar (https://docs.google.com/spreadsheets/d/18ROI6dRdKsNfXg5gIyBa1_7eYOjowfbw5n65zkrLnvc/edit#gid=1680005064), one doubling of consumption is about 0.4 DALYs

@dataclass
class Spreadsheet:
    name: str
    url: str
    result_cell_coordinate: str
    units: Units = Units.daly_over_dollar
    result_worksheet: str = ""

    def __hash__(self) -> int:
        return hash(self.name + self.url + self.result_cell_coordinate + self.result_worksheet)

In [29]:
spreadsheets = [
    Spreadsheet(
        "Technical assistance/advocacy on replacement of iron supplementation programs to MMN for pregnant mothers - BOTEC",
        "https://docs.google.com/spreadsheets/d/1342XLGqjpLXV4pHXINz2SrzAora2xhm-99x6slt64Xs/",
        "E72",
        Units.dollar_over_daly,
        "Daggered",
    ),
    Spreadsheet(
        "GiveWell IFA CEA for EvAc Beta India grant",
        "https://docs.google.com/spreadsheets/d/1_ttwAj4rH9rDhqGeil01hBmmrdP2qjIF2RvVMB5BDT8/edit#gid=0",
        "B96",
        Units.in_gd_daly_over_dollar,
        "Main",
    )
]

In [40]:
class Dagger:
    def __init__(self, spreadsheets, sensitivity=False):
        self.spreadsheets = spreadsheets
        self.sensitivity = sensitivity
        self.runs = {spreadsheet: {'id':"", 'result':None, 'status':"NOT_STARTED"} for spreadsheet in spreadsheets}
        
    def run(self):
        self.start_jobs()
        self.get_all_results()
    
    @staticmethod
    def spreadsheet_to_params(spreadsheet: Spreadsheet, sensitivity: bool):
        return {
            "url": spreadsheet.url,
            "result_worksheet": spreadsheet.result_worksheet,
            "result_cell_coordinate": spreadsheet.result_cell_coordinate,
            "sensitivity": sensitivity,
        }
    
    def start_jobs(self):
        post_url = 'https://usedagger.com/api/spreadsheet/'
        for spreadsheet in self.spreadsheets:
            params = self.spreadsheet_to_params(spreadsheet, self.sensitivity)
            result = requests.post(post_url, json=params)
            self.runs[spreadsheet]["id"] = result.json()['id']
            self.runs[spreadsheet]["status"] = "STARTED"

    @staticmethod
    def get_results(id):
        results = requests.get(f'https://usedagger.com/api/sim/{id}')
        status = results.json()['status']['status']
        if status == "SUCCESS":
            return status, pd.DataFrame(results.json()['output']['simulation_data'])
        return status, None
            
    def get_all_results(self, attempts=20, wait=5):
        for spreadsheet in self.spreadsheets:
            if self.runs[spreadsheet]['status'] == "SUCCESS":
                continue
            print(f"Getting results for {spreadsheet.name}")
            for attempt in range(attempts):
                status, result = self.get_results(self.runs[spreadsheet]['id'])
                self.runs[spreadsheet]['status'] = status
                if status == "SUCCESS":
                    self.runs[spreadsheet]['result'] = result
                    break
                if status == "STARTED":
                    time.sleep(wait)
                    continue
                print(f"Error: {status} for {spreadsheet.name}")
            if attempt == attempts - 1:
                print(f"Error: could not get results for {spreadsheet.name} after {attempts} attempts")
                self.runs[spreadsheet]['status'] = "TOO_LONG"
            print(f"Got results for {spreadsheet.name}, status: {self.runs[spreadsheet]['status']}")





In [41]:
run = Dagger(spreadsheets)

In [42]:
run.run()

Getting results for Technical assistance/advocacy on replacement of iron supplementation programs to MMN for pregnant mothers - BOTEC
Got results for Technical assistance/advocacy on replacement of iron supplementation programs to MMN for pregnant mothers - BOTEC, status: SUCCESS
Getting results for GiveWell IFA CEA for EvAc Beta India grant
Got results for GiveWell IFA CEA for EvAc Beta India grant, status: SUCCESS


In [43]:
run.runs


{Spreadsheet(name='Technical assistance/advocacy on replacement of iron supplementation programs to MMN for pregnant mothers - BOTEC', url='https://docs.google.com/spreadsheets/d/1342XLGqjpLXV4pHXINz2SrzAora2xhm-99x6slt64Xs/', result_cell_coordinate='E72', units=<function Units.<lambda> at 0x0000011C43F84040>, result_worksheet='Daggered'): {'id': '9anepd9Gn8MecfNqSbj7Nm',
  'result':       Daggered!E72  Daggered!E71  Daggered!E70  Daggered!E63  Daggered!E69   
  0        22.406702     22.406702     22.406702           0.0     22.406702  \
  1        26.591198     26.591198     26.591198           0.0     26.591198   
  2        21.885277     21.885277     21.885277           0.0     21.885277   
  3       102.073100    102.073100    102.073100           0.0    102.073100   
  4        13.577529     13.577529     13.577529           0.0     13.577529   
  ...            ...           ...           ...           ...           ...   
  2995     25.687618     25.687618     25.687618       